# XGBRegressor Pipeline

In [1]:
# Import Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import os
import pickle
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold, train_test_split, cross_val_score

Different brands and models were concatenated to one dataframe

In [2]:
# Read and concatenate all CSV's

path = r'D:\Data_Analytics\Project 3\Data'
all_files = glob.glob(path + "/*.csv")

li = []
brands = ["Audi","Skoda","BMW","Volkswagen","Toyota","Mercedes Benz","Ford","Hyundi"]

for filename, brand in zip(all_files, brands):
    df = pd.read_csv(filename, index_col=None, header=0)
    df["make"] = brand
    li.append(df)
    
frame = pd.concat(li, axis=0, ignore_index=True)
frame


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,make
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4,Audi
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0,Audi
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4,Audi
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0,Audi
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0,Audi
...,...,...,...,...,...,...,...,...,...,...
85550,I30,2016,8680,Manual,25906,Diesel,0,78.4,1.6,Hyundi
85551,I40,2015,7830,Manual,59508,Diesel,30,65.7,1.7,Hyundi
85552,I10,2017,6830,Manual,13810,Petrol,20,60.1,1.0,Hyundi
85553,Tucson,2018,13994,Manual,23313,Petrol,145,44.8,1.6,Hyundi


In [3]:
# remove unwanted feature
frame =frame.drop("tax", axis=1)

In [4]:
frame_filtered = frame.loc[frame['year'] != 2060]
frame_filtered

,model,year,price,transmission,mileage,fuelType,mpg,engineSize,make
0,A1,2017,12500,Manual,15735,Petrol,55.4,1.4,Audi
1,A6,2016,16500,Automatic,36203,Diesel,64.2,2.0,Audi
2,A1,2016,11000,Manual,29946,Petrol,55.4,1.4,Audi
3,A4,2017,16800,Automatic,25952,Diesel,67.3,2.0,Audi
4,A3,2019,17300,Manual,1998,Petrol,49.6,1.0,Audi
...,...,...,...,...,...,...,...,...,...
85550,I30,2016,8680,Manual,25906,Diesel,78.4,1.6,Hyundi
85551,I40,2015,7830,Manual,59508,Diesel,65.7,1.7,Hyundi
85552,I10,2017,6830,Manual,13810,Petrol,60.1,1.0,Hyundi
85553,Tucson,2018,13994,Manual,23313,Petrol,44.8,1.6,Hyundi


# Pipeline

Categorical and Numeric Features were preprocessed separately

In [5]:
# define numeric features

num_features = ['year','mileage','mpg','engineSize']

# Pipeline steps to transform numeric features
num_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant')),
                                      ('scaler', StandardScaler())])

In [6]:
# define categorical features

cat_features = ['model','transmission','fuelType','make']

# Pipeline to transform (encode) categorical features

cat_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [7]:
# Use "ColumnTransformer" to join both categorical and numeric pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)])

# Adriana's Step

In [ ]:
# transformer = Pipeline(steps=[('preprocessor', preprocessor)])

In [ ]:
# Define X, y

# X = frame_filtered.drop('price', axis = 1)
# y = frame_filtered['price']

In [ ]:
# transformed_axis = transformer.fit(X,y)
# transformed_axis

In [ ]:
# pkl_filename = "axis_transformed.pkl"
# with open(pkl_filename, 'wb') as file:
#     pickle.dump(transformed_axis, file)

# Continue ...

In [8]:
# define Lasso model
regressor_model = XGBRegressor()

In [9]:
# Create a pipeline that execute "preprocessor" and the Lasso model
regression = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor',regressor_model )])

In [10]:
# Define X, y

X = frame_filtered.drop('price', axis = 1)
y = frame_filtered['price']

In [11]:
# Split X,y into train and test

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [26]:
X_train

,model,year,transmission,mileage,fuelType,mpg,engineSize,make
40184,Up,2013,Manual,30442,Petrol,60.1,1.0,Volkswagen
22786,1 Series,2016,Semi-Auto,37753,Diesel,67.3,2.0,BMW
47489,Aygo,2018,Semi-Auto,8751,Petrol,67.3,1.0,Toyota
34666,Polo,2019,Manual,11,Diesel,57.7,1.6,Volkswagen
8148,A6,2010,Manual,42000,Diesel,49.6,2.0,Audi
...,...,...,...,...,...,...,...,...
6265,A3,2019,Semi-Auto,2308,Petrol,34.5,2.0,Audi
54886,C Class,2019,Semi-Auto,15275,Petrol,45.6,1.5,Mercedes Benz
76820,Fiesta,2015,Manual,18000,Petrol,47.9,1.6,Ford
860,A1,2020,Manual,556,Petrol,47.9,1.0,Audi


In [12]:
# search hyperparameters

regression.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'preprocessor', 'regressor', 'preprocessor__n_jobs', 'preprocessor__remainder', 'preprocessor__sparse_threshold', 'preprocessor__transformer_weights', 'preprocessor__transformers', 'preprocessor__verbose', 'preprocessor__num', 'preprocessor__cat', 'preprocessor__num__memory', 'preprocessor__num__steps', 'preprocessor__num__verbose', 'preprocessor__num__imputer', 'preprocessor__num__scaler', 'preprocessor__num__imputer__add_indicator', 'preprocessor__num__imputer__copy', 'preprocessor__num__imputer__fill_value', 'preprocessor__num__imputer__missing_values', 'preprocessor__num__imputer__strategy', 'preprocessor__num__imputer__verbose', 'preprocessor__num__scaler__copy', 'preprocessor__num__scaler__with_mean', 'preprocessor__num__scaler__with_std', 'preprocessor__cat__memory', 'preprocessor__cat__steps', 'preprocessor__cat__verbose', 'preprocessor__cat__imputer', 'preprocessor__cat__onehot', 'preprocessor__cat__imputer__add_indicator', 'preprocesso

In [13]:
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 40, num = 5)]
max_depth = [int(x) for x in np.linspace(2, 18, num = 10)]
max_depth.append(None)
gbm_param_grid = {
    'regressor__colsample_bytree': [0.3, 0.7],
    'regressor__n_estimators': n_estimators,
    'regressor__max_depth': max_depth
}


In [14]:
# apply GridSearchCV for tuning
search = GridSearchCV(estimator = regression, param_grid= gbm_param_grid, cv=4, verbose = 3)

In [15]:
# fit the model
search.fit(X_train, y_train)

Fitting 4 folds for each of 110 candidates, totalling 440 fits
[CV] regressor__colsample_bytree=0.3, regressor__max_depth=2, regressor__n_estimators=20 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  regressor__colsample_bytree=0.3, regressor__max_depth=2, regressor__n_estimators=20, score=0.832, total=   1.0s
[CV] regressor__colsample_bytree=0.3, regressor__max_depth=2, regressor__n_estimators=20 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV]  regressor__colsample_bytree=0.3, regressor__max_depth=2, regressor__n_estimators=20, score=0.832, total=   1.1s
[CV] regressor__colsample_bytree=0.3, regressor__max_depth=2, regressor__n_estimators=20 


...

[CV]  regressor__colsample_bytree=0.7, regressor__max_depth=14, regressor__n_estimators=20, score=0.957, total=   1.4s
[CV] regressor__colsample_bytree=0.7, regressor__max_depth=14, regressor__n_estimators=20 
[CV]  regressor__colsample_bytree=0.7, regressor__max_depth=14, regressor__n_estimators=20, score=0.959, total=   1.4s
[CV] regressor__colsample_bytree=0.7, regressor__max_depth=14, regressor__n_estimators=20 
[CV]  regressor__colsample_bytree=0.7, regressor__max_depth=14, regressor__n_estimators=20, score=0.964, total=   1.4s
[CV] regressor__colsample_bytree=0.7, regressor__max_depth=14, regressor__n_estimators=20 
[CV]  regressor__colsample_bytree=0.7, regressor__max_depth=14, regressor__n_estimators=20, score=0.959, total=   1.4s
[CV] regressor__colsample_bytree=0.7, regressor__max_depth=14, regressor__n_estimators=25 
[CV]  regressor__colsample_bytree=0.7, regressor__max_depth=14, regressor__n_estimators=25, score=0.958, total=   1.7s
[CV] regressor__colsample_bytree=0.7, reg

[CV]  regressor__colsample_bytree=0.7, regressor__max_depth=18, regressor__n_estimators=20, score=0.959, total=   1.8s
[CV] regressor__colsample_bytree=0.7, regressor__max_depth=18, regressor__n_estimators=20 
[CV]  regressor__colsample_bytree=0.7, regressor__max_depth=18, regressor__n_estimators=20, score=0.960, total=   1.8s
[CV] regressor__colsample_bytree=0.7, regressor__max_depth=18, regressor__n_estimators=20 
[CV]  regressor__colsample_bytree=0.7, regressor__max_depth=18, regressor__n_estimators=20, score=0.965, total=   1.8s
[CV] regressor__colsample_bytree=0.7, regressor__max_depth=18, regressor__n_estimators=20 
[CV]  regressor__colsample_bytree=0.7, regressor__max_depth=18, regressor__n_estimators=20, score=0.960, total=   1.8s
[CV] regressor__colsample_bytree=0.7, regressor__max_depth=18, regressor__n_estimators=25 
[CV]  regressor__colsample_bytree=0.7, regressor__max_depth=18, regressor__n_estimators=25, score=0.959, total=   2.1s
[CV] regressor__colsample_bytree=0.7, reg

[CV]  regressor__colsample_bytree=0.7, regressor__max_depth=None, regressor__n_estimators=40, score=0.942, total=   1.3s


[Parallel(n_jobs=1)]: Done 440 out of 440 | elapsed:  9.1min finished


GridSearchCV(cv=4, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('num',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('imputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=True,
                                    

In [16]:
# Print the best parameters and lowest RMSE
print("Best parameters found: ", search.best_params_)
print("Best Score found: ", np.sqrt(np.abs(search.best_score_)))

Best parameters found:  {'regressor__colsample_bytree': 0.7, 'regressor__max_depth': 16, 'regressor__n_estimators': 40}
Best Score found:  0.980391237590368


In [17]:
# check test accuracy
print('Test Acc: %.3f' % search.score(X_test, y_test))

Test Acc: 0.958


In [18]:
# check best score
print('Test Acc: %.3f' % search.best_score_)

Test Acc: 0.961


In [19]:
# define model "best params"
search.best_params_

{'regressor__colsample_bytree': 0.7,
 'regressor__max_depth': 16,
 'regressor__n_estimators': 40}

# Saving the Model

In [20]:
pkl_filename = "regression_XGB_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(search, file)

# Real Predictions

In [27]:
values = ['Yaris',2015,'Automatic', 36021,'Hybrid',78,1.5,'Toyota']

In [28]:
features = [np.array(values)]

In [29]:
df = pd.DataFrame(features,columns=['model','year','transmission','mileage','fuelType','mpg','engineSize','make'])
df

,model,year,transmission,mileage,fuelType,mpg,engineSize,make
0,Yaris,2015,Automatic,36021,Hybrid,78,1.5,Toyota


In [30]:
pkl_filename = "regression_XGB_model.pkl"
with open(pkl_filename, 'rb') as file:
    model_linear = pickle.load(file)

In [31]:
model_linear.predict(df)

array([12289.301], dtype=float32)